In [1]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.7 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score
from gensim.models import FastText

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tsel = pd.read_csv(r'/content/drive/MyDrive/TA/Prepro_Sinyal.csv', usecols=['Username','Text','Sinyal','text_result'])
tsel

,Username,Text,Sinyal,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",-1.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,-1.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,0.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",0.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",-1.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,-1.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,-1.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [5]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [6]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Sinyal,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",-1.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,-1.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,0.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",0.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",-1.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,-1.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,-1.0,admin sinyal telkomsel rumah daerah rawabunga ...


## Ektraksi Fitur TF-IDF 

In [14]:
vectors= TfidfVectorizer(ngram_range=(1,2), max_features = 1000)
vectors

TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

In [15]:
tfidf = vectors.fit_transform(tsel['text_result'].astype(str))
tfidf

<16991x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 116944 stored elements in Compressed Sparse Row format>

In [16]:
df_tfidf = tfidf.copy()

In [17]:
X = df_tfidf
y = tsel['Sinyal']

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=25)

### AdaBoost


In [19]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from sklearn.ensemble import AdaBoostClassifier
adBoost = AdaBoostClassifier()

## Ekspansi Fitur

In [21]:
# Load Model trained
model_exp = FastText.load('/content/drive/MyDrive/TA/model_ntwt/fasttext/Twitter_Berita.fasttext').wv

In [43]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 20) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [44]:
feature_ef = vectors.get_feature_names()

In [45]:
# Membuat dataframe data test & train
tsel_x_train = pd.DataFrame(x_train.todense(), columns = feature_ef)
tsel_x_test = pd.DataFrame(x_test.todense(), columns = feature_ef)

In [46]:
dfexp_x_train= feature_expansion(tsel_x_train, feature_ef)

100%|██████████| 1000/1000 [00:21<00:00, 45.96it/s]


In [47]:
dfexp_x_test = feature_expansion(tsel_x_test, feature_ef)

100%|██████████| 1000/1000 [00:14<00:00, 68.44it/s]


## Klasifikasi

In [ ]:
# class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
#     def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
#         self.alpha = alpha
#         self.max_iter = max_iter
#         self.C = C
#         self.svm_ = [] # fuggly

#     def fit(self, X, y):
#         X, y = check_X_y(X, y, 'csr')
#         _, n_features = X.shape

#         labelbin = LabelBinarizer()
#         Y = labelbin.fit_transform(y)
#         self.classes_ = labelbin.classes_
#         if Y.shape[1] == 1:
#             Y = np.concatenate((1 - Y, Y), axis=1)

#         # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
#         # so we don't have to cast X to floating point
#         Y = Y.astype(np.float64)

#         # Count raw events from data
#         n_effective_classes = Y.shape[1]
#         self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
#         self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
#                                  dtype=np.float64)
#         self._compute_ratios(X, Y)

#         # flugglyness
#         for i in range(n_effective_classes):
#             X_i = X.multiply(self.ratios_[i])
#             svm = LinearSVC(C=self.C, max_iter=self.max_iter)
#             Y_i = Y[:,i]
#             svm.fit(X_i, Y_i)
#             self.svm_.append(svm) 

#         return self

#     def predict(self, X):
#         n_effective_classes = self.class_count_.shape[0]
#         n_examples = X.shape[0]

#         D = np.zeros((n_effective_classes, n_examples))

#         for i in range(n_effective_classes):
#             X_i = X.multiply(self.ratios_[i])
#             D[i] = self.svm_[i].decision_function(X_i)
        
#         return self.classes_[np.argmax(D, axis=0)]
        
#     def _compute_ratios(self, X, Y):
#         """Count feature occurrences and compute ratios."""
#         if np.any((X.data if issparse(X) else X) < 0):
#             raise ValueError("Input X must be non-negative")

#         self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
#         normalize(self.ratios_, norm='l1', axis=1, copy=False)
#         row_calc = lambda r: np.log(np.divide(r, (1 - r)))
#         self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
#         check_array(self.ratios_)
#         self.ratios_ = sparse.csr_matrix(self.ratios_)

In [48]:
y_pred_exp = adBoost.fit(dfexp_x_train, y_train).predict(dfexp_x_test)
y_true = y_test

### Top 1 

In [28]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1858   52   19]
 [ 118 1255    2]
 [  77    5   13]]
              precision    recall  f1-score   support

        -1.0       0.91      0.96      0.93      1929
         0.0       0.96      0.91      0.93      1375
         1.0       0.38      0.14      0.20        95

    accuracy                           0.92      3399
   macro avg       0.75      0.67      0.69      3399
weighted avg       0.91      0.92      0.91      3399

Precision:  0.7479749558157883
Recall:  0.6709209141426543
F1:  0.6896256881261845
Accuracy:  0.9196822594880847


### Top 5

In [35]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1865   47   17]
 [ 119 1254    2]
 [  68    7   20]]
              precision    recall  f1-score   support

        -1.0       0.91      0.97      0.94      1929
         0.0       0.96      0.91      0.93      1375
         1.0       0.51      0.21      0.30        95

    accuracy                           0.92      3399
   macro avg       0.79      0.70      0.72      3399
weighted avg       0.92      0.92      0.92      3399

Precision:  0.793468501620763
Recall:  0.6964495011504915
F1:  0.7234108279274644
Accuracy:  0.923506913798176


### Top 10

In [42]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1846   64   19]
 [ 119 1252    4]
 [  72    7   16]]
              precision    recall  f1-score   support

        -1.0       0.91      0.96      0.93      1929
         0.0       0.95      0.91      0.93      1375
         1.0       0.41      0.17      0.24        95

    accuracy                           0.92      3399
   macro avg       0.75      0.68      0.70      3399
weighted avg       0.91      0.92      0.91      3399

Precision:  0.7542750527532069
Recall:  0.6786463439337304
F1:  0.6992712045653752
Accuracy:  0.9161518093556928


### Top 20

In [49]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1845   58   26]
 [ 120 1253    2]
 [  72    4   19]]
              precision    recall  f1-score   support

        -1.0       0.91      0.96      0.93      1929
         0.0       0.95      0.91      0.93      1375
         1.0       0.40      0.20      0.27        95

    accuracy                           0.92      3399
   macro avg       0.75      0.69      0.71      3399
weighted avg       0.91      0.92      0.91      3399

Precision:  0.7542835903236131
Recall:  0.6892422828597012
F1:  0.7098708729420241
Accuracy:  0.9170344218887908
